In [2]:
import pandas as pd

We have created a 20% overlap for the samples to be labeled to assess the inter-rated agreement. Since annotator vary for each label duplicate pair, we can't use the sklearn's solution as it assumes that raters are the same in each case. Averaging the sklearn's  cohen_kappa_score along the pairs is not a viable solution since if you only have one data point per rater pair, the Cohen's kappa score calculation will result in a warning because it expects more variation in the data (or will return 0 if values are different).

We have calculated the Cohen's kappa for the dataset with the (wrong) assumption that there are only two annotators.
Further we improved the calculation with averaging the pairs, for that we have used this [solution](https://towardsdatascience.com/inter-annotator-agreement-2f46c6d37bf3) instead.

In [235]:
# https://towardsdatascience.com/inter-annotator-agreement-2f46c6d37bf3
def cohen_kappa(ann1, ann2):
    """Computes Cohen kappa for pair-wise annotators.
    :param ann1: annotations provided by first annotator
    :type ann1: list
    :param ann2: annotations provided by second annotator
    :type ann2: list
    :rtype: float
    :return: Cohen kappa statistic
    """
    count = 0
    for an1, an2 in zip(ann1, ann2):
        if an1 == an2:
            count += 1
    A = count / len(ann1)  # observed agreement A (Po)

    uniq = set(ann1 + ann2)
    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = ann1.count(item)
        cnt2 = ann2.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E += count

    return round((A - E) / (1 - E), 4)

# Loading and preprocessing the data

In [346]:
df_labels = pd.read_json('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/labels_ready.json')

In [347]:
import json
def extract_class_pairs(json_lst):
    pairs = []
    for json_str in json_lst:
        question_title = json_str['title']
        answer_title = json_str['answer']['title']
        pairs.append([question_title,answer_title])
    return pairs

In [348]:
df_labels['pairs'] = df_labels['Label'].apply(lambda x: extract_class_pairs(x['classifications']))

In [349]:
df = df_labels[['Created By', 'pairs', 'External ID']]

In [267]:
value_counts = df['External ID'].value_counts()
unique_rows = df[df['External ID'].isin(value_counts[value_counts == 1].index)]
df.drop(unique_rows.index, inplace=True)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_11694/3128516445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(unique_rows.index, inplace=True)


In [350]:
for i in range(3):
    col_name = 'pair{}'.format(i+1)
    df[col_name] = df['pairs'].apply(lambda x: x[i] if len(x) > i else None)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_11694/262876240.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = df['pairs'].apply(lambda x: x[i] if len(x) > i else None)
/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_11694/262876240.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = df['pairs'].apply(lambda x: x[i] if len(x) > i else None)
/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_11694/262876240.py:3: SettingWithCopyWarning: 
A valu

In [351]:
df = df[~df['pair1'].isna()]
df.reset_index(inplace=True, drop=True)

In [352]:
text2 = df['pair2'].iloc[16]
text2

['Чи прирівнюються українці до неістот, тварин чи людей, позбавлених людських рис (частково або повністю)?',
 'ні']

In [353]:
text3 = df['pair3'].iloc[16]
text3

['Чи присутня в тексті емоційна оцінка українців?', 'ні, оцінка не присутня']

In [354]:
def replace_with(cell, replace_list):
    if cell is None:
        return replace_list
    else:
        return cell

In [355]:
df["Dehumanization"] = df.pair2.apply(lambda x: replace_with(x, text2)[1])

In [356]:
df["Emotion"] = df.pair3.apply(lambda x: replace_with(x, text3)[1])

In [357]:
df["Mention"] = df['pair1'].apply(lambda x: x[1])

In [358]:
df = df.sort_values(by='External ID')
df.reset_index(inplace=True, drop=True)

# First attempt to calculate with the assumption that annotators are same

In [198]:
df['Rater'] = ['Rater1' if i % 2 == 0 else 'Rater2' for i in range(len(df))]

In [ ]:
from sklearn.metrics import cohen_kappa_score

# define a function to calculate Cohen's kappa for a given column
def calculate_kappa(column_name):
    rater1_labels = df[column_name].loc[df['Rater'] == 'Rater1'].tolist()
    rater2_labels = df[column_name].loc[df['Rater'] == 'Rater2'].tolist()
    kappa = cohen_kappa_score(rater1_labels, rater2_labels)
    return kappa

In [202]:
# calculate Cohen's kappa for the 'Dehumanization' column
dehumanization_kappa = calculate_kappa('Dehumanization')
print("Cohen's kappa for Dehumanization: {:.2f}".format(dehumanization_kappa))

# repeat the above for the 'Emotion' and 'Mention' columns
emotion_kappa = calculate_kappa('Emotion')
print("Cohen's kappa for Emotion: {:.2f}".format(emotion_kappa))

mention_kappa = calculate_kappa('Mention')
print("Cohen's kappa for Mention: {:.2f}".format(mention_kappa))

Cohen's kappa for Dehumanization: 0.50
Cohen's kappa for Emotion: 0.49
Cohen's kappa for Mention: 0.64


# Correct calculation under the real conditions of different annotators

In [359]:
def calculate_cohens_kappa(df, col_name):
    # creating the data for question
    df_column = df[['External ID', 'Created By', col_name]]

    # identifying the overlapping samples and raters who labeled them
    overlapping_samples = df_column.groupby('External ID').filter(lambda x: len(x) > 1)
    unique_sample_ids = overlapping_samples['External ID'].unique()
    rater_pairs = []
    for sample_id in unique_sample_ids:
        raters = overlapping_samples[overlapping_samples['External ID'] == sample_id]['Created By'].tolist()
        rater_pairs.append(raters)

    # averaging the kappa scores for pairs
    kappa_scores = []
    for sample_id, rater_pair in zip(unique_sample_ids, rater_pairs):
        sample_data = overlapping_samples[overlapping_samples['External ID'] == sample_id]
        rater1_labels = sample_data[sample_data['Created By'] == rater_pair[0]][col_name].tolist()[0]
        rater2_labels = sample_data[sample_data['Created By'] == rater_pair[1]][col_name].tolist()[0]
        kappa = cohen_kappa(rater1_labels, rater2_labels)
        kappa_scores.append(kappa)
    mean_kappa_score = sum(kappa_scores) / len(kappa_scores)

    return mean_kappa_score

In [360]:
calculate_cohens_kappa(df, 'Dehumanization')

0.850657954545455

In [361]:
calculate_cohens_kappa(df, 'Mention')

0.9697505681818179

In [362]:
calculate_cohens_kappa(df, 'Emotion')

0.8473826704545456

# Labels export

In [491]:
import os
import pandas as pd

directory_path = "/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/dataset_shuffled"
file_list = []
for filename in os.listdir(directory_path):
    if filename.endswith(".txt"):
        with open(os.path.join(directory_path, filename), "r") as file:
            file_content = file.read()
        file_list.append({"text": file_content, "file_name": filename})

df_datarows = pd.DataFrame(file_list)

In [492]:
df_datarows.rename(columns={'file_name':'External ID'}, inplace=True)
df_datarows

,text,External ID
0,"Как известно, свинья везде грязь найдет, и укр...",row_1281.txt
1,"Тогда как многие украинцы, наоборот, слепо вер...",row_2950.txt
2,"О таком щедром и мирном соседе, как Россия, мо...",row_3496.txt
3,🇺🇦❌Денацификация по-винницки Тивривский сельс...,row_2788.txt
4,Укронацисты объявили войну Пушкину.,row_2944.txt
...,...,...
3638,"«Я вас, бл@дей, на этот корабль три года собир...",row_2791.txt
3639,Награжден орденом Мужества за отвагу и самоотв...,row_474.txt
3640,🇷🇺В районе населенного пункта Часов Яр уничтож...,row_2949.txt
3641,Участники конкурса американской армии на замен...,row_1298.txt


In [514]:
df_datarows[df_datarows['External ID'] == 'row_3210.txt']

,text,External ID
2016,"""Бандеровцы на Украине сегодня взяли все худше...",row_3210.txt


In [494]:
df_labels

,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Updated At,Seconds to Label,Seconds to Review,...,Is Benchmark,Benchmark Agreement,Benchmark ID,Dataset Name,Reviews,View Label,Has Open Issues,Skipped,DataRow Workflow Info,pairs
0,clemlh3ot5jlz07zg93fndqql,cleh5u16r1y1d077nda1a1qc8,https://storage.labelbox.com/cldvi451e3lej07ww...,"{'objects': [], 'classifications': [{'featureI...",kateryna.burovova@ucu.edu.ua,Dehumanization,2023-02-27T09:05:35.000Z,2023-02-27T09:05:36.000Z,36.784,30.751,...,0,-1,NaN,Dehumanization_final_dataset,[],https://editor.labelbox.com?project=cleh63iav1...,0,False,"{'taskName': 'Done', 'Workflow History': [{'ac...",[[Чи згадуються в тексті українці або щось укр...
1,clemlh51q9umq07za3cx6h7v9,cleh5u16r1y1h077n7sp91xy0,https://storage.labelbox.com/cldvi451e3lej07ww...,"{'objects': [], 'classifications': [{'featureI...",kateryna.burovova@ucu.edu.ua,Dehumanization,2023-02-27T09:05:41.000Z,2023-02-27T09:05:41.000Z,6.542,1.000,...,0,-1,NaN,Dehumanization_final_dataset,[],https://editor.labelbox.com?project=cleh63iav1...,0,False,"{'taskName': 'Done', 'Workflow History': [{'ac...",[[Чи згадуються в тексті українці або щось укр...
2,clemlhd635hv4070q0dndfk3h,cleh5u16r1y1l077n8445az5a,https://storage.labelbox.com/cldvi451e3lej07ww...,"{'objects': [], 'classifications': [{'featureI...",kateryna.burovova@ucu.edu.ua,Dehumanization,2023-02-27T09:05:53.000Z,2023-02-27T09:05:53.000Z,16.489,6.234,...,0,-1,NaN,Dehumanization_final_dataset,[],https://editor.labelbox.com?project=cleh63iav1...,0,False,"{'taskName': 'Done', 'Workflow History': [{'ac...",[[Чи згадуються в тексті українці або щось укр...
3,clemlhglg1vlr0711eh7rbxsf,cleh5u16r1y1p077n7yxhdfio,https://storage.labelbox.com/cldvi451e3lej07ww...,"{'objects': [], 'classifications': [{'featureI...",kateryna.burovova@ucu.edu.ua,Dehumanization,2023-02-27T09:06:07.000Z,2023-02-27T09:06:09.000Z,304.801,291.289,...,0,-1,NaN,Dehumanization_final_dataset,[],https://editor.labelbox.com?project=cleh63iav1...,0,False,"{'taskName': 'Done', 'Workflow History': [{'ac...",[[Чи згадуються в тексті українці або щось укр...
4,clemrc8wr1kas07xzew1n4sf3,cleh5u16s1y1x077nggra0vhg,https://storage.labelbox.com/cldvi451e3lej07ww...,"{'objects': [], 'classifications': [{'featureI...",snizannabotvin@gmail.com,Dehumanization,2023-02-27T11:54:12.000Z,2023-02-27T19:05:13.000Z,353.906,3.816,...,0,-1,NaN,Dehumanization_final_dataset,[],https://editor.labelbox.com?project=cleh63iav1...,0,False,"{'taskName': 'Done', 'Workflow History': [{'ac...",[[Чи згадуються в тексті українці або щось укр...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,clf6u3sp40gds07yt6gjlasf1,cleh5u19e38eu077z4r3e62qh,https://storage.labelbox.com/cldvi451e3lej07ww...,"{'objects': [], 'classifications': [{'featureI...",eugene.1martynyuk@gmail.com,Dehumanization,2023-03-13T14:22:20.000Z,2023-03-13T14:22:20.000Z,14.318,0.000,...,0,-1,NaN,Dehumanization_final_dataset,[],https://editor.labelbox.com?project=cleh63iav1...,0,False,{'taskId': '3817fcae-b9f0-4015-95c1-1afe41afc4...,[[Чи згадуються в тексті українці або щось укр...
4280,clf6wydkb0jff07zy1tig2pgk,cleh5u19eig9w078n33qp7j53,https://storage.labelbox.com/cldvi451e3lej07ww...,"{'objects': [], 'classifications': [{'featureI...",eugene.1martynyuk@gmail.com,Dehumanization,2023-03-13T14:22:24.000Z,2023-03-13T14:22:24.000Z,6.467,0.000,...,0,-1,NaN,Dehumanization_final_dataset,[],https://editor.labelbox.com?project=cleh63iav1...,0,False,{'taskId': '3817fcae-b9f0-4015-95c1-1afe41afc4...,[[Чи згадуються в тексті українці або щось укр...
4281,clf6wyjgi051507y3dy1x46sz,cleh5u19e38di077zboeo8ujg,https://storage.labelbox.com/cldvi451e3lej07ww...,"{'objects': [], 'classifications': [{'featureI...",eugene.1martynyuk@gmail.com,Dehumanization,2023-03-13T14:23:09.000Z,2023-03-13T14:23:09.000Z,42.807,0.000,...,0,-1,NaN,Dehumanization_final_dataset,[],https://editor.labelbox.com?project=cleh63iav1...,0,False,{'taskId': '3817fcae

In [495]:
df_final = df[['External ID', 'Dehumanization', 'Emotion', 'Mention', 'Created By']]

In [496]:
df_final

,External ID,Dehumanization,Emotion,Mention,Created By
0,row_0.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com
1,row_1.txt,ні,"ні, оцінка не присутня",ні,snizannabotvin@gmail.com
2,row_10.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com
3,row_100.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com
4,row_1000.txt,ні,"ні, оцінка не присутня",так,tutovadesign@gmail.com
...,...,...,...,...,...
4245,row_996.txt,ні,"так, присутня негативна",так,tutovadesign@gmail.com
4246,row_997.txt,ні,"так, присутня негативна",так,yevhen.marchenko91@gmail.com
4247,row_998.txt,так,"так, присутня негативна",так,tutovadesign@gmail.com
4248,row_998.txt,так,"ні, оцінка не присутня",так,yevhen.marchenko91@gmail.com


In [497]:
df_merged = pd.merge(df_final, df_datarows, on='External ID', how='left')

In [498]:
#bs check
df_merged[df_merged['Mention']=='ні']['Dehumanization'].unique()


array(['ні'], dtype=object)

In [499]:
#bs check
df_merged[df_merged['Mention']=='ні']['Emotion'].unique()

array(['ні, оцінка не присутня'], dtype=object)

In [500]:
df_merged

,External ID,Dehumanization,Emotion,Mention,Created By,text
0,row_0.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,"Всвязи с этим немного поправлю коллег ⤵️ ""Они..."
1,row_1.txt,ні,"ні, оцінка не присутня",ні,snizannabotvin@gmail.com,Литературный критик Галина Юзефович о новом ро...
2,row_10.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,Почему на базах неонацистов стоят языческие ис...
3,row_100.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,Группа добровольцев-медиков из Чеченской Респу...
4,row_1000.txt,ні,"ні, оцінка не присутня",так,tutovadesign@gmail.com,"ВСУшники, переходите на сторону добра, у нас т..."
...,...,...,...,...,...,...
4245,row_996.txt,ні,"так, присутня негативна",так,tutovadesign@gmail.com,И понеслась мазепинщино-петлюровщино-бандеровщ...
4246,row_997.txt,ні,"так, присутня негативна",так,yevhen.marchenko91@gmail.com,Наш соратник по русскому движению Алексей Сели...
4247,row_998.txt,так,"так, присутня негативна",так,tutovadesign@gmail.com,Хорошее видео от 4 бригады НМ ЛНР https://t.me...
4248,row_998.txt,так,"ні, оцінка не присутня",так,yevhen.marchenko91@gmail.com,Хорошее видео от 4 бригады НМ ЛНР https://t.me...


At this stage we should choose whose labels should remain in the final dataset for those samples labeled by 2 labelers.

We have investigated a random portion of disagreements manually as only manual investigation can provide insights into potential biases and/or misunderstanding of guidelines.
We have also considered the annotator expertise and background that we were informed of.

In [501]:
df_labels['Created By'].unique()

array(['kateryna.burovova@ucu.edu.ua', 'snizannabotvin@gmail.com',
       'nazariy.melnychuk9@gmail.com', 'chennnakal@gmail.com',
       's.sterpul@icloud.com', 'mariana.scorp@gmail.com',
       'tutovadesign@gmail.com', 'yevhen.marchenko91@gmail.com',
       'eugene.1martynyuk@gmail.com'], dtype=object)

In [502]:
authors = df_labels['Created By'].unique().tolist()
rating = [1, 4, 7, 6, 8, 2, 5, 3, 9]

In [503]:
data = {'Created By': authors, 'rating': rating}
df_rating = pd.DataFrame(data)

In [504]:
df_rating

,Created By,rating
0,kateryna.burovova@ucu.edu.ua,1
1,snizannabotvin@gmail.com,4
2,nazariy.melnychuk9@gmail.com,7
3,chennnakal@gmail.com,6
4,s.sterpul@icloud.com,8
5,mariana.scorp@gmail.com,2
6,tutovadesign@gmail.com,5
7,yevhen.marchenko91@gmail.com,3
8,eugene.1martynyuk@gmail.com,9


In [505]:
df_combined = df_merged.merge(df_rating, on='Created By')

In [506]:
df_combined = df_combined.sort_values(['External ID', 'rating'], ascending=True)

In [507]:
df_combined

,External ID,Dehumanization,Emotion,Mention,Created By,text,rating
0,row_0.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,"Всвязи с этим немного поправлю коллег ⤵️ ""Они...",4
1,row_1.txt,ні,"ні, оцінка не присутня",ні,snizannabotvin@gmail.com,Литературный критик Галина Юзефович о новом ро...,4
2,row_10.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,Почему на базах неонацистов стоят языческие ис...,4
3,row_100.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,Группа добровольцев-медиков из Чеченской Респу...,4
1441,row_1000.txt,ні,"ні, оцінка не присутня",так,tutovadesign@gmail.com,"ВСУшники, переходите на сторону добра, у нас т...",5
...,...,...,...,...,...,...,...
2639,row_996.txt,ні,"так, присутня негативна",так,tutovadesign@gmail.com,И понеслась мазепинщино-петлюровщино-бандеровщ...,5
2760,row_997.txt,ні,"так, присутня негативна",так,yevhen.marchenko91@gmail.com,Наш соратник по русскому движению Алексей Сели...,3
2761,row_998.txt,так,"ні, оцінка не присутня",так,yevhen.marchenko91@gmail.com,Хорошее видео от 4 бригады НМ ЛНР https://t.me...,3
2640,row_998.txt,так,"так, присутня негативна",так,tutovadesign@gmail.com,Хорошее видео от 4 бригады НМ ЛНР https://t.me...,5


In [508]:
df_cleaned = df_combined.drop_duplicates(subset='External ID', keep='first')

In [509]:
df_cleaned

,External ID,Dehumanization,Emotion,Mention,Created By,text,rating
0,row_0.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,"Всвязи с этим немного поправлю коллег ⤵️ ""Они...",4
1,row_1.txt,ні,"ні, оцінка не присутня",ні,snizannabotvin@gmail.com,Литературный критик Галина Юзефович о новом ро...,4
2,row_10.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,Почему на базах неонацистов стоят языческие ис...,4
3,row_100.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,Группа добровольцев-медиков из Чеченской Респу...,4
1441,row_1000.txt,ні,"ні, оцінка не присутня",так,tutovadesign@gmail.com,"ВСУшники, переходите на сторону добра, у нас т...",5
...,...,...,...,...,...,...,...
2638,row_995.txt,ні,"ні, оцінка не присутня",так,tutovadesign@gmail.com,Утренний брифинг Минобороны России: ▪️ россий...,5
2639,row_996.txt,ні,"так, присутня негативна",так,tutovadesign@gmail.com,И понеслась мазепинщино-петлюровщино-бандеровщ...,5
2760,row_997.txt,ні,"так, присутня негативна",так,yevhen.marchenko91@gmail.com,Наш соратник по русскому движению Алексей Сели...,3
2761,row_998.txt,так,"ні, оцінка не присутня",так,yevhen.marchenko91@gmail.com,Хорошее видео от 4 бригады НМ ЛНР https://t.me...,3


Double-checking since it's crucial for our task

In [510]:
value_counts = df_combined['External ID'].value_counts()
unique_rows = df_combined[df_combined['External ID'].isin(value_counts[value_counts == 1].index)]
df_bs_check = df_combined.drop(unique_rows.index)

In [511]:
df_bs_check.sort_values(['External ID'])

,External ID,Dehumanization,Emotion,Mention,Created By,text,rating
2642,row_1005.txt,так,"так, присутня негативна",так,yevhen.marchenko91@gmail.com,"Оно провалилось, но укровояки хвалились тем, ч...",3
1446,row_1005.txt,ні,"так, присутня негативна",так,tutovadesign@gmail.com,"Оно провалилось, но укровояки хвалились тем, ч...",5
2643,row_1006.txt,так,"так, присутня негативна",так,yevhen.marchenko91@gmail.com,Западный мем о степени правдивости пропаганды ...,3
1447,row_1006.txt,так,"так, присутня негативна",так,tutovadesign@gmail.com,Западный мем о степени правдивости пропаганды ...,5
2644,row_1008.txt,так,"так, присутня негативна",так,yevhen.marchenko91@gmail.com,Война до последнего украинца – это вовсе не вы...,3
...,...,...,...,...,...,...,...
3809,row_98.txt,так,"так, присутня негативна",так,nazariy.melnychuk9@gmail.com,Благодаря такому единству можно уверенно гаран...,7
3997,row_984.txt,ні,"ні, оцінка не присутня",ні,kateryna.burovova@ucu.edu.ua,Этапы Китайской компартии по пути к мировой ге...,1
2627,row_984.txt,ні,"ні, оцінка не присутня",ні,tutovadesign@gmail.com,Этапы Китайской компартии по пути к мировой ге...,5
2761,row_998.txt,так,"ні, оцінка не присутня",так,yevhen.marchenko91@gmail.com,Хорошее видео от 4 бригады НМ ЛНР https://t.me...,3


In [512]:
df_cleaned[df_cleaned['External ID']=='row_1128.txt']

,External ID,Dehumanization,Emotion,Mention,Created By,text,rating
98,row_1128.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,По виртуальным планам укрорейха они уже взяли ...,4


In [516]:
df_cleaned.to_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/final_labels.csv')

In [515]:
df_cleaned[df_cleaned['External ID']=='row_3210.txt']

,External ID,Dehumanization,Emotion,Mention,Created By,text,rating
941,row_3210.txt,так,"так, присутня негативна",так,snizannabotvin@gmail.com,"""Бандеровцы на Украине сегодня взяли все худше...",4
